In [1]:
import re
import json
import nltk
import spacy
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gensim.downloader as api
import matplotlib.pyplot as plt

In [2]:
# Load the training data
# with open('/content/drive/MyDrive/data/train.json', 'r') as file:
with open('train.json', 'r') as file:
    data = json.load(file)
    train_data = pd.DataFrame(data['data'], columns=data['columns'])

# Load the test data
# with open('/content/drive/MyDrive/data/test.json', 'r') as file:
with open('test.json', 'r') as file:
    data = json.load(file)
    test_data = pd.DataFrame(data['data'], columns=data['columns'])

# Load the validation data
# with open('/content/drive/MyDrive/data/val.json', 'r') as file:
with open('val.json', 'r') as file:
    data = json.load(file)
    val_data = pd.DataFrame(data['data'], columns=data['columns'])

# Get the x and y lists for training data and test data
training_x = train_data['sentence'].tolist() + val_data['sentence'].tolist()
training_a = train_data['aspect'].tolist() + val_data['aspect'].tolist()
training_y = train_data['polarity'].tolist() + val_data['polarity'].tolist()
# output_y = [["<BOS>"] + [y] for y in training_y]
# target_y = [[y] + ["<EOS>"]for y in training_y]
test_x = test_data['sentence'].tolist()
test_a = test_data['aspect'].tolist()
test_y = test_data['polarity'].tolist()

In [3]:
! python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 393.8 kB/s eta 0:00:33
      --------------------------------------- 0.2/12.8 MB 1.4 MB/s eta 0:00:10
     - -------------------------------------- 0.5/12.8 MB 2.7 MB/s eta 0:00:05
     -- ------------------------------------- 0.7/12.8 MB 3.3 MB/s eta 0:00:04
     --- ------------------------------------ 1.0/12.8 MB 3.7 MB/s eta 0:00:04
     --- ------------------------------------ 1.3/12.8 MB 4.0 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 4.1 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 4.1 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 4.1 MB/s eta 0:00:03
     ----- ---------------------------------- 1.7/12.8 MB 3.4 MB/s eta 0:00:04
     ----- ---------------------------------- 1.9/12.8 MB 3.4 MB/

In [4]:
# Punctuation Removal
def remove_punctuation_re(x):
    x = re.sub(r'[^\w\s]', '', x)
    return x

nltk.download('punkt')

# Stopwords Removal
nlp = spacy.load("en_core_web_sm")

def remove_stopwords(x):
    doc = nlp(x)
    x = [word.text for word in doc if not word.is_stop]
    return x

# Stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def stem(x):
    x = [stemmer.stem(word) for word in x]
    return x

# Lemmatisation
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize(x):
    x = [lemmatizer.lemmatize(word) for word in x]
    return x

# English Contractions Dictionary
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have",
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

# Emoticons
EMOTICONS = r"""
    (?:
      [<>]?
      [:;=8]                     # eyes
      [\-o\*\']?                 # optional nose
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
      |
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
      [\-o\*\']?                 # optional nose
      [:;=8]                     # eyes
      [<>]?
      |
      <3                         # heart
    )"""

EMOTICON_RE = re.compile(EMOTICONS, re.VERBOSE | re.I | re.UNICODE)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\allis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\allis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# Function to preprocess the data
def preprocess_data(sentence_list):
    output_list = []
    for sentence in sentence_list:
        emoticons = EMOTICON_RE.findall(sentence)       # Extract emoticons
        sentence = EMOTICON_RE.sub('', sentence)        # Remove emoticons
        sentence = sentence.lower()                     # Case folding
        for word, new_word in contraction_dict.items(): # Deal with contractions
            sentence = sentence.replace(word, new_word)
        sentence = remove_punctuation_re(sentence)      # Remove punctuation
        tokens = remove_stopwords(sentence)             # Remove stopwords / Tokenise
        # tokens = stem(tokens)                           # Stemming
        # tokens = lemmatize(tokens)                      # Lemmatisation
        tokens.extend(emoticons)                        # Add emoticons back
        output_list.append(tokens)
    return output_list

# Preprocess the data and get the tokenised sentence lists
train_x_token = preprocess_data(training_x)
test_x_token = preprocess_data(test_x)

In [7]:
def list_to_index(token_list, word_to_idx):
    idx_list = []
    for tokens in token_list:
        idx_list.append([word_to_idx[token] for token in tokens])
    return idx_list

def string_to_index(aspect_list, word_to_idx):
    idx_list = []
    for aspect in aspect_list:
        idx_list.append(word_to_idx[aspect])
    return idx_list

# Token index lists for training data
train_x_idx = list_to_index(train_x_token, word_to_idx)
train_a_idx = string_to_index(training_a, word_to_idx)
train_y_idx = string_to_index(training_y, polarity_to_idx)

In [6]:
# Get longest sentence length
longest_sentence = max(train_x_token, key=len)
MAX_LENGTH = len(longest_sentence)

# Word vocabulary to index dictionary {word: index}
word_to_idx = {'<PAD>': 0, '<SEP>': 1}
review = train_x_token + test_x_token
aspect = [list(set(training_a))]
polarity = [list(set(training_y))]

# Vocabulary includes words from reviews and aspects
for sentence in aspect+review:
    for word in sentence:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
word_list = list(word_to_idx.keys())
idx_to_word = {idx: word for word, idx in word_to_idx.items()}
vocab_size = len(word_to_idx)

# Polarity vocabulary to index dictionary {polarity: index}
polarity_to_idx = {p: i for i, p in enumerate(polarity[0])}
idx_to_polarity = {idx: pol for pol, idx in polarity_to_idx.items()}

In [8]:
# Encoder
class GRUEncoder(nn.Module):
    def __init__(self, hidden_size, embedding):
        super(GRUEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, input, hidden, aspect):
        word_embedding = self.embedding(input).view(1, 1, -1)
        aspect_embedding = self.embedding(aspect).view(1, 1, -1)
        word_embedding = torch.cat((word_embedding, aspect_embedding), 0)
        output, hidden = self.gru(word_embedding, hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size)

# Decoder
class GRUDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, attention='dot_product'):
        super(GRUDecoder, self).__init__()
        self.attention_type = attention
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.linear = nn.Linear(self.hidden_size*2, self.output_size)
    
    def cal_attention(self, hidden, encoder_hiddens):
        if self.attention_type == 'dot_product':            # Dot product attention
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)), dim=-1)
        elif self.attention_type == 'scaled_dot_product':   # Scaled dot product attention
            scale = 1.0 / np.sqrt(self.hidden_size)
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)) * scale, dim=-1)
        elif self.attention_type == 'cosine_similarity':
            query = hidden / torch.norm(hidden, dim=-1)
            keys = encoder_hiddens / torch.norm(encoder_hiddens.T, dim=-1)
            norm_product = torch.bmm(query, keys.T.unsqueeze(0))
            attn_weights = F.softmax(norm_product, dim=-1)
        attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
        concat_output = torch.cat((attn_output[0], hidden[0]), 1)
        return concat_output
    
    def forward(self, hidden, encoder_hiddens):
        concat_output = self.cal_attention(hidden, encoder_hiddens)
        output = F.log_softmax(self.linear(concat_output), dim=1)
        return output

In [10]:
# Set the seed for reproducibility
# seed = 4012
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# np.random.seed(seed)
# random.seed(seed)

num_epochs = 10000
display_interval = 500
learning_rate = 0.01
hidden_size = 50
attention_type = 'dot_product' # 'dot_product', 'scaled_dot_product', 'cosine_similarity'
embedding = nn.Embedding(vocab_size, hidden_size)
encoder = GRUEncoder(hidden_size, embedding)
decoder = GRUDecoder(hidden_size, 3, attention=attention_type)

encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()
plot_losses = []
plot_avg_losses = []
total_loss = 0

for epoch in range(1, num_epochs+1):
    random_idx = random.choice(range(len(train_data)))
    x_index = [[idx] for idx in train_x_idx[random_idx]]
    y_index = train_y_idx[random_idx]
    a_index = train_a_idx[random_idx]
    
    x_tensor = torch.LongTensor(x_index)
    y_tensor = torch.LongTensor([y_index])
    a_tensor = torch.LongTensor([a_index])
    x_length = x_tensor.size(0)
    y_length = y_tensor.size(0)
    
    loss = 0
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    # Feed the x_tensor (sentence) into the encoder
    encoder_hiddens = torch.zeros(MAX_LENGTH, encoder.hidden_size) # For attention mechanism
    encoder_hidden = encoder.init_hidden() # Hidden state for encoder
    for i in range(x_length):
        encoder_output, encoder_hidden = encoder(x_tensor[i], encoder_hidden, a_tensor)
        encoder_hiddens[i] = encoder_hidden[0, 0]

    y_output = decoder(encoder_hidden, encoder_hiddens)
    loss += criterion(y_output, y_tensor)

    # Backpropagation
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    loss = loss.item() / y_length
    total_loss += loss
    plot_losses.append(loss)
    
    if (epoch+1) % display_interval == 0:
        avg_loss = total_loss / display_interval
        plot_avg_losses.append(avg_loss)
        total_loss = 0
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

Epoch [500/10000], Loss: 1.1066
Epoch [1000/10000], Loss: 1.2869
Epoch [1500/10000], Loss: 1.2461
Epoch [2000/10000], Loss: 1.2791
Epoch [2500/10000], Loss: 1.2392
Epoch [3000/10000], Loss: 1.3278
Epoch [3500/10000], Loss: 1.3357
Epoch [4000/10000], Loss: 1.2343
Epoch [4500/10000], Loss: 1.3251
Epoch [5000/10000], Loss: 1.2514
Epoch [5500/10000], Loss: 1.2851
Epoch [6000/10000], Loss: 1.3019
Epoch [6500/10000], Loss: 1.4022
Epoch [7000/10000], Loss: 1.3596
Epoch [7500/10000], Loss: 1.3119
Epoch [8000/10000], Loss: 1.3099
Epoch [8500/10000], Loss: 1.3161
Epoch [9000/10000], Loss: 1.2568
Epoch [9500/10000], Loss: 1.3188
Epoch [10000/10000], Loss: 1.2272


In [13]:
# Plot loss over number of epochs
plt.plot(range(1, num_epochs+1), plot_losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over number of epochs')
plt.show()

plt.plot(range(1, num_epochs+1, display_interval), plot_avg_losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over number of epochs')
plt.show()

In [11]:
# GRU evaluate
def GRUevaluate(encoder, decoder, sentence, aspect, max_length=MAX_LENGTH):
    with torch.no_grad():
        input = preprocess_data([sentence])[0]
        input_idx = [word_to_idx[word] for word in input]
        input_tensor = torch.LongTensor([[ind] for ind in input_idx])
        
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.init_hidden()
        
        asp_idx = word_to_idx[aspect]
        asp_tensor = torch.LongTensor([[asp_idx]])

        encoder_hiddens = torch.zeros(max_length, encoder.hidden_size)

        for ei in range(input_length):
            _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden, asp_tensor)
            encoder_hiddens[ei] += encoder_hidden[0, 0]

        y_pred = decoder(encoder_hidden, encoder_hiddens)
        topv, topi = y_pred.data.topk(1)
        return idx_to_polarity[topi.item()]

In [12]:
# Measure accuracy of GRU model on test set
correct = 0
for i in range(len(test_x)):
    polarity = GRUevaluate(encoder, decoder, test_x[i], test_a[i])
    if polarity == test_y[i]:
        correct += 1
accuracy = correct / len(test_x_token)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.5250


In [ ]:
# Encoder
class Encoder(nn.Module):
    def __init__(self, hidden_size, embedding):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, input, hidden, aspect):
        word_embedding = self.embedding(input).view(1, 1, -1)
        aspect_embedding = self.embedding(aspect).view(1, 1, -1)
        word_embedding = torch.cat((aspect_embedding, word_embedding), 0)
        output, hidden = self.gru(word_embedding, hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size)

# Decoder
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, attention='dot_product'):
        super(Decoder, self).__init__()
        self.attention_type = attention
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.linear = nn.Linear(self.hidden_size*2, self.output_size)
    
    def cal_attention(self, hidden, encoder_hiddens):
        if self.attention_type == 'dot_product':            # Dot product attention
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)), dim=-1)
        elif self.attention_type == 'scaled_dot_product':   # Scaled dot product attention
            scale = 1.0 / np.sqrt(self.hidden_size)
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)) * scale, dim=-1)
        elif self.attention_type == 'cosine_similarity':
            query = hidden / torch.norm(hidden, dim=-1)
            keys = encoder_hiddens / torch.norm(encoder_hiddens.T, dim=-1)
            norm_product = torch.bmm(query, keys.T.unsqueeze(0))
            attn_weights = F.softmax(norm_product, dim=-1)

        attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
        concat_output = torch.cat((attn_output[0], hidden[0]), 1)
        return concat_output
    
    def forward(self, hidden, encoder_hiddens):
        concat_output = self.cal_attention(hidden, encoder_hiddens)
        output = F.log_softmax(self.linear(concat_output), dim=1)
        return output
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [ ]:
# Set the seed for reproducibility
# seed = 4012
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# np.random.seed(seed)
# random.seed(seed)

num_epochs = 10000
display_interval = 500
learning_rate = 0.01
hidden_size = 50
attention_type = 'dot_product' # 'dot_product', 'scaled_dot_product', 'cosine_similarity'

# embedding = nn.Embedding.from_pretrained(glove_weights)
embedding = nn.Embedding(vocab_size, hidden_size)

encoder = Encoder(hidden_size, embedding)
decoder = Decoder(hidden_size, 3, attention=attention_type)

encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()
plot_losses = []
plot_avg_losses = []
total_loss = 0

for epoch in range(1, num_epochs+1):
    random_idx = random.choice(range(len(train_data)))
    x_index = [[idx] for idx in train_x_idx[random_idx]]
    y_index = train_y_idx[random_idx]
    a_index = train_a_idx[random_idx]
    
    x_tensor = torch.LongTensor(x_index)
    y_tensor = torch.LongTensor([y_index])
    a_tensor = torch.LongTensor([a_index])
    x_length = x_tensor.size(0)

    loss = 0
    encoder.train()
    decoder.train()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    # Feed the x_tensor (sentence) into the encoder
    encoder_hiddens = torch.zeros(MAX_LENGTH, encoder.hidden_size) # For attention mechanism
    encoder_hidden = encoder.init_hidden() # Hidden state for encoder
    for i in range(x_length):
        encoder_output, encoder_hidden = encoder(x_tensor[i], encoder_hidden, a_tensor)
        encoder_hiddens[i] = encoder_hidden[0, 0]

    y_output = decoder(encoder_hidden, encoder_hiddens)
    loss += criterion(y_output, y_tensor)

    # Backpropagation
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    loss = loss.item()
    total_loss += loss
    plot_losses.append(loss)
    
    if (epoch+1) % display_interval == 0:
        avg_loss = total_loss / display_interval
        plot_avg_losses.append(avg_loss)
        total_loss = 0
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

In [ ]:
# Save the model
# torch.save(encoder, 'Model/encoder1.pt')
# torch.save(decoder, 'Model1/decoder1.pt')

# Plot loss over number of epochs
plt.plot(range(1, num_epochs+1), plot_losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over number of epochs')
plt.show()

plt.plot(range(1, num_epochs+1, display_interval), plot_avg_losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over number of epochs')
plt.show()

In [ ]:
# GRU evaluate
def GRUevaluate(encoder, decoder, sentence, aspect, max_length=MAX_LENGTH):
    with torch.no_grad():
        input = preprocess_data([sentence])[0]
        input_idx = [word_to_idx[word] for word in input]
        input_tensor = torch.LongTensor([[ind] for ind in input_idx])
        
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.init_hidden()
        
        asp_idx = word_to_idx[aspect]
        asp_tensor = torch.LongTensor([[asp_idx]])

        encoder_hiddens = torch.zeros(max_length, encoder.hidden_size)

        for ei in range(input_length):
            _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden, asp_tensor)
            encoder_hiddens[ei] += encoder_hidden[0, 0]
        
        y_pred = decoder(encoder_hidden, encoder_hiddens)
        topv, topi = y_pred.data.topk(1)
        return idx_to_polarity[topi.item()]

In [ ]:
# Measure accuracy of LSTM model on test set
correct = 0
for i in range(len(test_x)):
    polarity = GRUevaluate(encoder, decoder, test_x[i], test_a[i])
    if polarity == test_y[i]:
        correct += 1
accuracy = correct / len(test_x_token)
print(f"Accuracy: {accuracy:.4f}")